# Add custom generators and metrics
Benchmarking requires a common API, where generators have a fit() and generate() method and metrics have a compute() method. You can add custom generators and metrics by subclassing the BaseGenerator and BaseMetric classes, so these are interoperable with the benchmarking framework.

In [1]:
import numpy as np
import pandas as pd
import warnings

In [14]:
# ignore warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# autoreload changes from local files
%load_ext autoreload
%autoreload 2

# pandas show full output
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from crnsynth.serialization import paths
from crnsynth.benchmark.benchmark import benchmark_generators
from crnsynth.benchmark.review import SyntheticDataReview
from crnsynth.metrics.privacy.dcr import DistanceClosestRecord
from crnsynth.generators.marginal import MarginalGenerator
from crnsynth.generators.base import BaseGenerator
from crnsynth.metrics.base import BaseMetric
from crnsynth.processing.preprocessing import split_train_holdout

## Add custom generator

For example here we add the CTGAN generator from SDV.

In [11]:
from sdv.single_table import CTGANSynthesizer

class CTGANGenerator(BaseGenerator):
    def __init__(self, metadata):
        self.generator = CTGANSynthesizer(metadata)
    
    def fit(self, real_data):
        self.generator.fit(real_data)

    def generate(self, n):
        return self.generator.sample(n)

In [12]:
from sdv.datasets.demo import download_demo

df, metadata = download_demo(
    modality='single_table',
    dataset_name='fake_hotel_guests'
)

df_train, df_holdout = split_train_holdout(df, holdout_size=0.2)
df_train.head()

,guest_email,has_rewards,room_type,amenities_fee,checkin_date,checkout_date,room_rate,billing_address,credit_card_number
120,jaywheeler@pratt.com,True,DELUXE,0.00,01 Jun 2020,04 Jun 2020,225.69,"27260 Amy Grove Suite 845\nBensonborough, PA 5...",4189467192702831
114,bryan94@jennings-reyes.com,True,DELUXE,0.00,09 Mar 2020,10 Mar 2020,141.94,"922 Karen Inlet\nTheresachester, DE 47978",38883115614070
429,fjenkins@dawson-kelly.com,False,SUITE,15.80,02 Mar 2020,05 Mar 2020,286.76,"1234 Corporate Drive\nBoston, MA 02116",3522235794160338
68,noahvargas@gallegos.net,False,SUITE,25.64,05 Jan 2020,07 Jan 2020,251.15,"0549 Lynn Hill\nSimmonsland, MI 00790",372770926213149
220,keithkim@cooke.com,False,DELUXE,29.40,21 Sep 2020,24 Sep 2020,192.77,"1234 Corporate Drive\nBoston, MA 02116",370010845439463


In [15]:
generator = CTGANGenerator(metadata=metadata)
generator.fit(df_train)
df_synth = generator.generate(1000)
df_synth.head()

,guest_email,has_rewards,room_type,amenities_fee,checkin_date,checkout_date,room_rate,billing_address,credit_card_number
0,masseycaroline@example.net,True,SUITE,25.72,26 Oct 2020,22 Oct 2020,100.96,"81292 William Port\nNew Rodney, OK 92198",30100763314715
1,tsimmons@example.org,False,BASIC,48.12,17 Oct 2020,11 Apr 2020,180.69,"PSC 7842, Box 3562\nAPO AP 06457",585442349734
2,erik18@example.com,False,DELUXE,10.64,05 Jan 2020,15 Jun 2020,144.26,"3668 Chapman Vista\nLake Kara, UT 72584",3588053650969887
3,gina17@example.org,True,BASIC,11.37,09 Sep 2020,07 Jan 2020,119.99,"5482 Jasmine Crossroad Suite 796\nJeffreyton, ...",4841975543771
4,tiffanyprice@example.net,False,BASIC,NaN,04 Aug 2020,17 Aug 2020,141.07,"148 Ashley Skyway\nSouth Herbertshire, OK 29438",4911205436349810


## Add custom metric

In [16]:
class AverageCardinalityPreserved(BaseMetric):
    """Compute percentage of cardinality of categorical columns in synthetic data compared to real data."""

    
    def __init__(self, categorical_columns=None):
        self.categorical_columns = categorical_columns
        
        self.scores_ = {}
    
    @staticmethod
    def type() -> str:
        return 'similarity'
    
    @staticmethod
    def direction() -> str:
        return 'maximize'
    
    def compute(self, real_data, synthetic_data, holdout=None):
        if self.categorical_columns is None:
            self.categorical_columns = real_data.select_dtypes(include='object').columns.tolist()
        
        cardinality_scores = np.zeros(len(self.categorical_columns))
        for i, col in enumerate(self.categorical_columns):
            # get unique values of each column
            unique_real = real_data[col].unique()
            unique_synth = synthetic_data[col].unique()
            
            # calculate percentage of categories from real data that are in the synth data
            percentage_overlap = len(set(unique_real).intersection(set(unique_synth))) / len(unique_real)
            cardinality_scores[i] = percentage_overlap
            
        # take average of all columns
        self.scores_['score'] = np.mean(cardinality_scores)
        return self.scores_
    
metric_cardinality = AverageCardinalityPreserved(categorical_columns=['has_rewards', 'room_type'])
metric_cardinality.compute(df_train, df_synth)
                            
    

{'score': 1.0}

Metrics can also be imported from other libraries. These just need to have the compute() method. For the popular libraries, like `synthcity`, we created a wrapper class that allows you to import the metric and use it in the benchmarking framework.

In [38]:
from crnsynth.metrics.wrappers import SynthcityMetricWrapper
from synthcity.metrics.eval_statistical import JensenShannonDistance
    
sc_js = SynthcityMetricWrapper(metric=JensenShannonDistance(), include_holdout=False, encoder='ordinal')
sc_js.compute(df_train, df_synth, df_holdout)

{'marginal': 0.06229167482897598}

## Benchmark

In [39]:
reviewer = SyntheticDataReview(
    metrics=[AverageCardinalityPreserved(), DistanceClosestRecord(quantile=0.05), sc_js],
    metric_kwargs = {'categorical_columns': ['has_rewards', 'room_type']}
)

benchmark_generators(
    data_real=df_train,
    data_holdout=df_holdout,    
    generators=[MarginalGenerator(epsilon=0.1), CTGANGenerator(metadata=metadata)],
    reviewer=reviewer,
    path_out = paths.PATH_RESULTS / 'fake_hotel_guests',
)

Running generator MarginalGenerator
Fitting generator MarginalGenerator on input data
Marginal fitted: guest_email
Marginal fitted: has_rewards
Marginal fitted: room_type
Marginal fitted: amenities_fee
Marginal fitted: checkin_date
Marginal fitted: checkout_date
Marginal fitted: room_rate
Marginal fitted: billing_address
Marginal fitted: credit_card_number
Generator fitted. Generating 400 records
Column sampled: guest_email
Column sampled: has_rewards
Column sampled: room_type
Column sampled: amenities_fee
Column sampled: checkin_date
Column sampled: checkout_date
Column sampled: room_rate
Column sampled: billing_address
Column sampled: credit_card_number
Saved to disk: /Users/dknoors/Projects/synthesis-dk/crn-synth/results/fake_hotel_guests/configs/0_MarginalGenerator.json
Saved synthetic data, generator and configs for 0_MarginalGenerator at /Users/dknoors/Projects/synthesis-dk/crn-synth/results/fake_hotel_guests
Running reviewer for 0_MarginalGenerator
Running metric AverageCardinal

In [40]:
# load results
scores_benchmark = pd.read_csv(paths.PATH_RESULTS / 'fake_hotel_guests/reports/scores.csv')
scores_benchmark

,metric,0_MarginalGenerator,1_CTGANGenerator
0,AverageCardinalityPreserved_score,1.000000,1.000000
1,DistanceClosestRecord_holdout,1.000000,1.000000
2,DistanceClosestRecord_synth,0.004670,1.000000
3,JensenShannonDistance_marginal,0.012365,0.062057


In [20]:
from crnsynth.processing.encoding import encode_data

df_enc, encoders = encode_data(df_train, encoder='onehot')
df_enc.head()

,guest_email_aaron27@rowland-miller.net,guest_email_aarondiaz@williams.com,guest_email_abigail37@wheeler.com,guest_email_adamsmark@phillips-barnes.com,guest_email_adavis@lee.com,guest_email_ajenkins@smith-horton.org,guest_email_albertperez@james-simmons.com,guest_email_alexander84@stewart.com,guest_email_alexanderlaura@hutchinson.com,guest_email_alexandriaholloway@hart.com,guest_email_alexis29@garcia.com,guest_email_alisonblackburn@houston.info,guest_email_allenpowell@cook.com,guest_email_allisonhowe@mcdonald-roberts.com,guest_email_amanda09@nelson.com,guest_email_amanda14@klein-mcconnell.net,guest_email_amandasmith@cox-galvan.com,guest_email_amcintosh@nicholson.com,guest_email_andrew80@bush.com,guest_email_andrewarcher@turner.com,guest_email_angelawalker@bray.biz,guest_email_ann70@jones.com,guest_email_anthony26@mosley.com,guest_email_ashley21@gonzales.info,guest_email_asmith@jenkins.com,guest_email_austinhunter@flores-bennett.com,guest_email_autumnbrown@black.net,guest_email_bakerjennifer@holmes.com,guest_email_barbara31@frazier.com,guest_email_baxtertami@flores-allison.com,guest_email_bbrown@newton.com,guest_email_belljose@goodwin-farrell.net,guest_email_blakemichael@elliott.info,guest_email_blewis@mills.com,guest_email_boyerkyle@mcbride.com,guest_email_bradley48@jones-saunders.net,guest_email_bradley71@kennedy.biz,guest_email_brandon12@baxter.org,guest_email_brandondodson@washington-melendez.com,guest_email_brightjessica@evans.net,guest_email_brittany47@joseph.com,guest_email_brittanybailey@bates-williams.com,guest_email_bryan94@jennings-reyes.com,guest_email_bryanmack@hansen.com,guest_email_bunderwood@miller.com,guest_email_bushandrew@freeman-fox.info,guest_email_bvalenzuela@torres.com,guest_email_caleb90@baker.org,guest_email_carmen23@hodges.org,guest_email_carsonnathan@singleton.com,guest_email_carterselena@zavala.com,guest_email_cassandramiller@sheppard.com,guest_email_castrojeffrey@clark.com,guest_email_castromelissa@scott-flores.com,guest_email_catherine16@allen.com,guest_email_cathynelson@robles-williams.com,guest_email_cbutler@sanchez.org,guest_email_chad98@price.com,guest_email_chambersaaron@stanley-potts.org,guest_email_charles01@sullivan-jackson.org,guest_email_charlesmendoza@montgomery.org,guest_email_chelsea98@compton.biz,guest_email_christina28@pearson.net,guest_email_christine53@carson.org,guest_email_christineperry@carlson.com,guest_email_christophergarcia@harris.com,guest_email_christopherlane@knight.com,guest_email_colin60@rivas-foster.com,guest_email_colleenharvey@barnett.com,guest_email_coreytaylor@porter-choi.com,guest_email_courtney10@mack-mccormick.com,guest_email_courtneyclark@young.com,guest_email_coxdiana@brewer.com,guest_email_craiglawson@wilson.com,guest_email_cvasquez@gutierrez-smith.net,guest_email_cwilliams@snow-hill.biz,guest_email_cyates@hodge-mays.com,guest_email_cynthia01@knight.com,guest_email_danielreid@perry.biz,guest_email_danieltaylor@harper.com,guest_email_darrellswanson@ross-wilson.net,guest_email_davery@moore-thompson.com,guest_email_david29@house.com,guest_email_david55@marshall.org,guest_email_david92@nichols.com,guest_email_david95@torres.com,guest_email_daybarbara@graham.com,guest_email_dean26@garza.com,guest_email_deanna47@king-king.com,guest_email_deborah36@escobar.net,guest_email_destiny51@underwood.biz,guest_email_dford@cole-chang.com,guest_email_dicksonjames@george-hughes.com,guest_email_douglasayers@mcclain.net,guest_email_doylebeth@terrell.com,guest_email_dpennington@colon.org,guest_email_dscott@anderson-hatfield.info,guest_email_dterry@steele-guerrero.com,guest_email_duketara@jenkins.org,guest_email_dustin75@pierce.com,...,credit_card_number_4484017750569544,credit_card_number_4500509902359766,credit_card_number_4501453483101314,credit_card_number_4534892217780995,credit_card_number_4561422003901286,credit_card_number_4595200078272209,credit_card_number_4606786888323064,credit_card_number_4628655862037822,credit_card_number_4650088697745024,credit_card_number_465474454